In [7]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# -----------------------------
# Config (edit these if needed)
# -----------------------------
CSV_PATH = "ga_year_demo_3_projects.csv"   # path to your dataset
OUT_DIR = Path("./outputs")                # where results will be saved
SHOW_CHARTS = False                        # True to display charts inline (e.g., in notebooks)


# -----------------------------
# Helper functions
# -----------------------------
def aggregate_funnel(g: pd.DataFrame) -> pd.Series:
    """Aggregate a group (or full DF) into funnel & efficiency KPIs."""
    total = {
        "impressions": g["impressions"].sum(),
        "clicks": g["clicks"].sum(),
        "sessions": g["sessions"].sum(),
        "transactions": g["transactions"].sum(),
        "revenue": g["revenue"].sum(),
        "ad_spend": g["ad_spend"].sum()
    }
    # Derived metrics
    total["ctr"] = (total["clicks"] / total["impressions"]) if total["impressions"] > 0 else np.nan
    total["click_to_session"] = (total["sessions"] / total["clicks"]) if total["clicks"] > 0 else np.nan
    total["conversion_rate"] = (total["transactions"] / total["sessions"]) if total["sessions"] > 0 else np.nan
    total["cpc"] = (total["ad_spend"] / total["clicks"]) if total["clicks"] > 0 else np.nan
    total["cpm"] = (total["ad_spend"] / (total["impressions"] / 1000)) if total["impressions"] > 0 else np.nan
    total["cpa"] = (total["ad_spend"] / total["transactions"]) if total["transactions"] > 0 else np.nan
    total["roas"] = (total["revenue"] / total["ad_spend"]) if total["ad_spend"] > 0 else np.nan
    return pd.Series(total)


def plot_funnel_bar(values, title, outpath: Path | None = None, show: bool = False):
    """Bar chart of funnel stages."""
    stages = ["Impressions", "Clicks", "Sessions", "Transactions"]
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(stages, values)
    ax.set_title(title)
    ax.set_ylabel("Count")
    ax.set_xlabel("Funnel Stage")
    for i, v in enumerate(values):
        try:
            label = f"{int(v):,}"
        except Exception:
            label = "-"
        ax.text(i, v, label, ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    if outpath:
        fig.savefig(outpath, dpi=200, bbox_inches="tight")
    if show:
        plt.show()
    plt.close(fig)


def plot_monthly_line(df_proj: pd.DataFrame, metric: str, ylabel: str, title: str,
                      outpath: Path | None = None, show: bool = False):
    """Monthly trend lines per project."""
    fig, ax = plt.subplots(figsize=(7, 4))
    for proj, g in df_proj.groupby("project"):
        g = g.sort_values("month")
        ax.plot(g["month"], g[metric], label=proj)
    ax.set_title(title)
    ax.set_xlabel("Month")
    ax.set_ylabel(ylabel)
    ax.legend()
    plt.tight_layout()
    if outpath:
        fig.savefig(outpath, dpi=200, bbox_inches="tight")
    if show:
        plt.show()
    plt.close(fig)


# -----------------------------
# Main analysis runner
# -----------------------------
def run_analysis(csv_path: str | Path, out_dir: str | Path, show_charts: bool = False):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load data
    df = pd.read_csv(csv_path, parse_dates=["date"])
    required_cols = {"project", "impressions", "clicks", "sessions", "transactions", "revenue", "ad_spend"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {', '.join(sorted(missing))}")

    # Month for grouping
    df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

    # Summaries
    overall = aggregate_funnel(df)
    by_project = df.groupby("project").apply(aggregate_funnel).reset_index()

    # Save summaries
    overall.to_frame().T.to_csv(out_dir / "funnel_summary_overall.csv", index=False)
    by_project.to_csv(out_dir / "funnel_summary_by_project.csv", index=False)

    # Monthly KPIs by project
    monthly = df.groupby(["month", "project"]).agg({
        "impressions": "sum",
        "clicks": "sum",
        "sessions": "sum",
        "transactions": "sum",
        "revenue": "sum",
        "ad_spend": "sum"
    }).reset_index()

    monthly["ctr"] = np.where(monthly["impressions"] > 0, monthly["clicks"] / monthly["impressions"], np.nan)
    monthly["click_to_session"] = np.where(monthly["clicks"] > 0, monthly["sessions"] / monthly["clicks"], np.nan)
    monthly["conversion_rate"] = np.where(monthly["sessions"] > 0, monthly["transactions"] / monthly["sessions"], np.nan)
    monthly["cpc"] = np.where(monthly["clicks"] > 0, monthly["ad_spend"] / monthly["clicks"], np.nan)
    monthly["cpm"] = np.where(monthly["impressions"] > 0, monthly["ad_spend"] / (monthly["impressions"] / 1000), np.nan)
    monthly["cpa"] = np.where(monthly["transactions"] > 0, monthly["ad_spend"] / monthly["transactions"], np.nan)
    monthly["roas"] = np.where(monthly["ad_spend"] > 0, monthly["revenue"] / monthly["ad_spend"], np.nan)

    monthly.to_csv(out_dir / "monthly_funnel_by_project.csv", index=False)

    # Charts
    plot_funnel_bar(
        [overall["impressions"], overall["clicks"], overall["sessions"], overall["transactions"]],
        "Overall Funnel",
        out_dir / "overall_funnel.png",
        show=show_charts
    )

    for _, row in by_project.iterrows():
        vals = [row["impressions"], row["clicks"], row["sessions"], row["transactions"]]
        safe_name = str(row["project"]).replace(" ", "_")
        plot_funnel_bar(
            vals,
            f"Funnel — {row['project']}",
            out_dir / f"funnel_{safe_name}.png",
            show=show_charts
        )

    # Monthly trends
    plot_monthly_line(monthly, "sessions", "Sessions", "Monthly Sessions by Project",
                      out_dir / "trend_sessions.png", show=show_charts)
    plot_monthly_line(monthly, "transactions", "Transactions", "Monthly Transactions by Project",
                      out_dir / "trend_transactions.png", show=show_charts)
    plot_monthly_line(monthly, "ad_spend", "Ad Spend (€)", "Monthly Ad Spend by Project",
                      out_dir / "trend_ad_spend.png", show=show_charts)
    plot_monthly_line(monthly, "revenue", "Revenue (€)", "Monthly Revenue by Project",
                      out_dir / "trend_revenue.png", show=show_charts)
    plot_monthly_line(monthly, "ctr", "CTR", "Monthly CTR by Project",
                      out_dir / "trend_ctr.png", show=show_charts)
    plot_monthly_line(monthly, "conversion_rate", "Conversion Rate", "Monthly Conversion Rate by Project",
                      out_dir / "trend_conversion_rate.png", show=show_charts)

    print(f"Done. CSVs and charts saved to: {out_dir.resolve()}")


# -----------------------------
# Run immediately (script mode)
# -----------------------------
if __name__ == "__main__":
    # If you’re running this as a script, it will execute with the defaults above.
    if not Path(CSV_PATH).exists():
        raise FileNotFoundError(
            f"Could not find '{CSV_PATH}'. Put your GA CSV next to this script or update CSV_PATH."
        )
    run_analysis(CSV_PATH, OUT_DIR, show_charts=SHOW_CHARTS)


C:\Users\USER\AppData\Local\Temp\ipykernel_9244\773864051.py:113: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_project = df.groupby("project").apply(aggregate_funnel).reset_index()


Done. CSVs and charts saved to: C:\Users\USER\Documents\Python files\GA\outputs
